In [1]:
import os 
cwd = os.getcwd()
os.chdir(cwd.rsplit("/",1)[0])
from geneformer import TranscriptomeTokenizer
from geneformer import EmbExtractor

/rsrch5/home/plm/phacosta/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-12-11 23:38:01.233520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 23:38:01.273435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 23:38:01.273628: E external/local_xla/xla/stream_execut

In [2]:
task = "fine_tune" # fine_tune or pretrained

In [ ]:
cancer = "lung"
xenium_folder_dict = {"breast": "Xenium_Prime_Human_Lung_Cancer_FFPE_outs",
                      "lung":"Xenium_Prime_Human_Lung_Cancer_FFPE_outs",
                      "lymph_node": "Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs",
                      "prostate": "Xenium_Prime_Human_Prostate_FFPE_outs",
                      "skin": "Xenium_Prime_Human_Skin_FFPE_outs",
                      "ovarian": "Xenium_Prime_Ovarian_Cancer_FFPE_outs"
                      }

xenium_folder = xenium_folder_dict[cancer]

data_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/public_data/10x_genomics/{xenium_folder}"

# Use x_scaled and y_scaled as custom attributes
custom_attributes = {
    "x_scaled": "x_scaled",
    "y_scaled": "y_scaled",
    "total_counts": "total_counts",  # Include normalized transcript counts
    "target" : "class"
}
tk = TranscriptomeTokenizer(custom_attributes, nproc=16)

# Run the tokenizer
tk.tokenize_data(
    data_path,
    os.path.join(data_path, "tokenize_output"),
    "processed_xenium",
    file_format="h5ad"
)


In [ ]:
embex = EmbExtractor(model_type="Pretrained",
                     emb_layer=-1,
                     forward_batch_size=100,
                     max_ncells=None,
                     nproc=16)

# extracts embedding from input data
# input data is tokenized rank value encodings generated by Geneformer tokenizer (see tokenizing_scRNAseq_data.ipynb)
 
embs = embex.extract_embs("gf-12L-95M-i4096",
                          os.path.join(data_path, "tokenize_output", "processed_xenium.dataset"),
                          os.path.join(data_path, "embeddings_output"),
                          "processed_xenium")